In [1]:
from bs4 import BeautifulSoup as BS
import re
from time import sleep
import requests
from yandexfreetranslate import YandexFreeTranslate
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm

In [2]:
# name = 'i-have-a-mansion-in-the-post-apocalyptic-world'
name = 'super-gene'

In [3]:
url = 'https://novelbjn.novelupdates.net/ajax/chapter-option'

params = {
    'novelId': name,
}

res_links = requests.get(url=url, params=params)
page = BS(res_links.text, 'html.parser')
links = list(map(lambda x: x.get('value'), page.find('body').find_all('option')))


In [16]:
links[-37]

'https://novelbjn.novelupdates.net/book/i-have-a-mansion-in-the-post-apocalyptic-world/chapter-1573'

In [5]:
len(links)-len(links[-37:])

1475

In [4]:
links = list(map(lambda x: re.sub(r'(.*:\/\/.*?\/)', 'https://novelbjn.novelmagic.org/', x), links))

In [5]:
book = []

In [6]:
start = 0
start_book = len(links) - len(links[start:])
for url in tqdm(links[start:]):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    }

    res_page = requests.get(
        url,
        headers=headers,
    )
    page = BS(res_page.text, 'html.parser').find('div', id='chr-content')
    

    
    paragraphs = [tag.text.strip() for tag in page.find('div').next_siblings if tag.name in ['p', 'h4', 'h3']]
    book.append(paragraphs)

  0%|          | 0/3462 [00:00<?, ?it/s]

In [20]:
def _worker(pool_list: tuple[str, tqdm]) -> str:
    queue, pbar = pool_list
    if not queue.strip():
        pbar.update(1)
        return ''
    yt = YandexFreeTranslate(api = "ios")
    
    for _ in range(10):
        try:
            text = yt.translate("en", "ru", queue)
        except Exception:
            sleep(0.5)
            continue
        else:
            break
    else:
        text = queue
        pbar.write(str(queue))
    pbar.update(1)
    return text
    

def run_translate(lst: list[str], pbar: tqdm)-> list[str]:
    pbar.reset(len(lst))
    pool_list = list(zip(lst, [pbar]*len(lst)))
    with ThreadPool(8) as pool:
        workreturn = pool.map(_worker, pool_list)
    return workreturn

In [21]:
new_book = []

In [22]:
pbar1 = tqdm(desc='page')
pbar2 = tqdm(desc='row')
pbar1.reset(len(links[start:]))
for index, page in list(enumerate(book, start=start_book)):
    try:
        new_page = run_translate(page, pbar2)
        new_book.append(new_page)
    except Exception:
        print(index)
        break
    pbar1.update()
pbar1.close()
pbar2.close()
    

page: 0it [00:00, ?it/s]

row: 0it [00:00, ?it/s]

Not because of self-confidence, but because the security work in place was perfectly safe. Even if she wasn’t there, there were other sisters of Ghost Agents accompanying him. And the most crucial thing was that, even if one set aside all these failsafes, Jiang Chen’s own strength was no match for the average person.
The clock just struck 10. Nightlife had only just begun for Coro City.
Ne/w novel chapt𝒆rs are p/u/blished on no/vel(/bin(.)c/o/m
“Mhmm.”
Silence continued for a moment. Natasha went directly to the subject and put on a serious expression.
“You didn’t come back with him?”
It was already his fifth cup.
“You don’t seem worried?”
There was still work to be done and she was not good at drinking...
It’s funny to say that this was Jiang Chen’s first time coming to the club... if the time in the Sixth Street doesn’t count.
What’s more, this was his new world.
“Based on KGB’s intelligence, some people are plotting an assassination against him.”
At this moment, a man dressed in a p

In [ ]:
from ebooklib import epub
from random import randint

# Функция для преобразования страницы в HTML
def page_to_html(page, title):
    content_lines = page[:]
    # Формирование HTML-кода
    html = f"<h1>{title}</h1>"
    for line in content_lines:
        html += f"<p>{line}</p>"
    return html


title = name

# Создание новой книги
epub_book = epub.EpubBook()

# Установка метаданных
epub_book.set_identifier(f"id{randint(100000,1000000)}")
epub_book.set_title(title)
epub_book.set_language("ru")

# Автор
epub_book.add_author("None")
# epub_book.spine.append('nav')
# Обработка каждой страницы
for i, page in enumerate(new_book, start=start_book):
    # Создание главы
    chapter_title = f'Глава {i+1:04d}'
    chapter_content = page_to_html(page, chapter_title)
    
    chapter = epub.EpubHtml(title=chapter_title, file_name=f'chap_{i+1:04d}.xhtml', lang='ru')
    chapter.content = chapter_content
    
    epub_book.add_item(chapter)
    
    # Добавление в оглавление
    epub_book.toc.append(chapter)
    # Упаковка всех глав в книгу
    epub_book.spine.append(chapter)


# Добавление оглавления
epub_book.add_item(epub.EpubNcx())
epub_book.add_item(epub.EpubNav())

# Сохранение книги
epub.write_epub(f'{name}.epub', epub_book)
